In [51]:
from pymongo import MongoClient
import requests as rq
import datetime as dt

cardiac = [
    'http://nosql.hpeixoto.me/api/sensor/3001',
    'http://nosql.hpeixoto.me/api/sensor/3002',
    'http://nosql.hpeixoto.me/api/sensor/3003',
    'http://nosql.hpeixoto.me/api/sensor/3004',
    'http://nosql.hpeixoto.me/api/sensor/3005'
]

blood = [
    'http://nosql.hpeixoto.me/api/sensor/4001',
    'http://nosql.hpeixoto.me/api/sensor/4002',
    'http://nosql.hpeixoto.me/api/sensor/4003',
    'http://nosql.hpeixoto.me/api/sensor/4004',
    'http://nosql.hpeixoto.me/api/sensor/4005'
]

url = "mongodb://localhost:27017"
client = MongoClient(url)
db = client.sensors

# Only the first result matters
def getLastTimestamp(col):
    m = col.find({},{"_id":0, "timestamp": 1}).sort("_id",-1).limit(1)
    for ts in m:
        return ts['timestamp']

# Only insert if the timestamp is more recent than the last on the db
def maybeInsert(col,data,tsdb,tsjson):
    if(tsdb >= tsjson):
        return False
    else:
        col.insert_one(data)
        return True

# Insert into db = sensors, collection = sensorYYYY
def parseJson(data,type):
    print(">> Currently on sensor: type = {0}, ID = {1}".format(type, data['sensorid']))
    id = str(data['sensorid'])
    col = db[type + id]
    date = getLastTimestamp(col)
    result = maybeInsert(col,data,date,data['timestamp']) 
    print(data['timestamp'] + ' > ' + date + ' ?', "Yes, adding the sensor data to the DB.\n" if result else "No, skipping the current sensor data.\n")

for sensor in cardiac:
    data = rq.get(sensor).json()
    parseJson(data, 'cardiac')

for sensor in blood:
    data = rq.get(sensor).json()
    parseJson(data, 'blood')


>> Currently on sensor: type = cardiac, ID = 3001
2020-11-15 22:12:45 > 2020-11-15 22:12:38 ? Yes, adding the sensor data to the DB.

>> Currently on sensor: type = cardiac, ID = 3002
2020-11-15 22:12:45 > 2020-11-15 22:12:39 ? Yes, adding the sensor data to the DB.

>> Currently on sensor: type = cardiac, ID = 3003
2020-11-15 22:12:45 > 2020-11-15 22:12:39 ? Yes, adding the sensor data to the DB.

>> Currently on sensor: type = cardiac, ID = 3004
2020-11-15 22:12:45 > 2020-11-15 22:12:39 ? Yes, adding the sensor data to the DB.

>> Currently on sensor: type = cardiac, ID = 3005
2020-11-15 22:12:45 > 2020-11-15 22:12:39 ? Yes, adding the sensor data to the DB.

>> Currently on sensor: type = blood, ID = 4001
2020-11-15 22:12:47 > 2020-11-15 22:12:38 ? Yes, adding the sensor data to the DB.

>> Currently on sensor: type = blood, ID = 4002
2020-11-15 22:12:47 > 2020-11-15 22:12:38 ? Yes, adding the sensor data to the DB.

>> Currently on sensor: type = blood, ID = 4003
2020-11-15 22:12:5